## Problem Statement 

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling 

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [1]:
import json 
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [4]:
# Opening JSON file 
f = open('/kaggle/input/automatic-ticket-classification/complaints-2021-05-14_08_16.json')# Write the path to your data file and load it 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [5]:
# Inspect the dataframe to understand the given data.
df.sample(10)


,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
57446,complaint-public-v2,complaint,2600747,0.0,None,900XX,2600747,Closing on a mortgage,2017-08-10T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-08-10T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
2989,complaint-public-v2,complaint,29845,0.0,None,75126,29845,"Loan servicing, payments, escrow account",2012-03-04T12:00:00-05:00,TX,...,Closed without relief,JPMORGAN CHASE & CO.,Web,2012-03-05T12:00:00-05:00,None,Conventional fixed mortgage,Yes,,None,N/A
59682,complaint-public-v2,complaint,491740,0.0,None,902XX,491740,"Loan servicing, payments, escrow account",2013-08-13T12:00:00-05:00,CA,...,Closed,JPMORGAN CHASE & CO.,Referral,2013-08-14T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
13187,complaint-public-v2,complaint,1697069,0.0,None,31210,1697069,"Account opening, closing, or management",2015-12-15T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2015-12-16T12:00:00-05:00,None,Checking account,Yes,,None,N/A
63897,complaint-public-v2,complaint,30642,0.0,None,073XX,30642,Credit determination,2012-03-06T12:00:00-05:00,NJ,...,Closed with relief,JPMORGAN CHASE & CO.,Web,2012-03-06T12:00:00-05:00,None,None,Yes,,None,N/A
10628,complaint-public-v2,complaint,3518424,0.0,None,088XX,3518424,Trouble during payment process,2020-01-31T12:00:00-05:00,NJ,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2020-02-05T12:00:00-05:00,None,Other type of mortgage,Yes,,None,N/A
61120,complaint-public-v2,complaint,977060,0.0,None,28104,977060,"Loan modification,collection,foreclosure",2014-08-10T12:00:00-05:00,NC,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2014-08-13T12:00:00-05:00,None,Conventional fixed mortgage,Yes,,None,N/A
542,complaint-public-v2,complaint,3448018,0.0,None,None,3448018,Problem with a company's investigation into an...,2019-11-23T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-11-23T12:00:00-05:00,None,Other personal consumer report,Yes,"On XX/XX/2019, I walked inside the JP Morgan C...",Problem with personal statement of dispute,Consent provided
15949,complaint-public-v2,complaint,2657515,0.0,None,None,2657515,"Other features, terms, or problems",2017-09-03T12:00:00-05:00,MA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-03T12:00:00-05:00,None,Store credit card,Yes,I applied for an Amazon credit card ( Chase Ba...,Other problem,Consent provided
6709,complaint-public-v2,complaint,4272966,0.0,Servicemember,953XX,4272966,Problem with a purchase shown on your statement,2021-04-05T12:00:00-05:00,CA,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2021-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Credit card company isn't resolving a dispute ...,None


In [6]:
df.shape

(78313, 22)

In [7]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [8]:
#Assign new column names
for column in df.columns:
    new_column = column.replace("_","").replace("source.", "")
    df.rename(columns={column: new_column}, inplace=True)

In [9]:
df.columns

Index(['index', 'type', 'id', 'score', 'tags', 'zipcode', 'complaintid',
       'issue', 'datereceived', 'state', 'consumerdisputed', 'product',
       'companyresponse', 'company', 'submittedvia', 'datesenttocompany',
       'companypublicresponse', 'subproduct', 'timely',
       'complaintwhathappened', 'subissue', 'consumerconsentprovided'],
      dtype='object')

In [10]:
df.sample(10)

,index,type,id,score,tags,zipcode,complaintid,issue,datereceived,state,...,companyresponse,company,submittedvia,datesenttocompany,companypublicresponse,subproduct,timely,complaintwhathappened,subissue,consumerconsentprovided
63989,complaint-public-v2,complaint,103595,0.0,None,10977,103595,"Loan modification,collection,foreclosure",2012-06-18T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2012-06-25T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
5196,complaint-public-v2,complaint,3673348,0.0,None,301XX,3673348,Improper use of your report,2020-05-29T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2020-05-29T12:00:00-05:00,None,Credit reporting,Yes,on XX/XX/XXXX I recived a inquiry on my credit...,Credit inquiries on your report that you don't...,Consent provided
22909,complaint-public-v2,complaint,2851123,0.0,None,None,2851123,Problem with a purchase shown on your statement,2018-03-22T12:00:00-05:00,None,...,Closed with explanation,JPMORGAN CHASE & CO.,Postal mail,2018-03-22T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Credit card company isn't resolving a dispute ...,N/A
36268,complaint-public-v2,complaint,101281,0.0,None,20876,101281,"Loan servicing, payments, escrow account",2012-06-13T12:00:00-05:00,MD,...,Closed,JPMORGAN CHASE & CO.,Referral,2012-06-18T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
61477,complaint-public-v2,complaint,2124349,0.0,None,945XX,2124349,Billing disputes,2016-09-21T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2016-09-21T12:00:00-05:00,None,None,Yes,I went to XXXX XXXX XX/XX/XXXX-XX/XX/2016 stay...,None,Consent provided
47945,complaint-public-v2,complaint,2925053,0.0,None,347XX,2925053,Managing an account,2018-06-03T12:00:00-05:00,FL,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2018-06-03T12:00:00-05:00,None,Checking account,Yes,I had a transaction posted to my account on XX...,Fee problem,Consent provided
54661,complaint-public-v2,complaint,1565552,0.0,None,None,1565552,"Loan servicing, payments, escrow account",2015-09-15T12:00:00-05:00,MN,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2015-09-15T12:00:00-05:00,None,Conventional adjustable mortgage (ARM),Yes,XX/XX/XXXX I received a notice from Chase that...,None,Consent provided
2582,complaint-public-v2,complaint,587025,0.0,None,088XX,587025,"Account opening, closing, or management",2013-11-05T12:00:00-05:00,NJ,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Referral,2013-11-07T12:00:00-05:00,None,Checking account,Yes,,None,N/A
26875,complaint-public-v2,complaint,4246327,0.0,None,850XX,4246327,Managing an account,2021-03-25T12:00:00-05:00,AZ,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2021-03-26T12:00:00-05:00,None,Checking account,Yes,,Deposits and withdrawals,None
54478,complaint-public-v2,complaint,1344584,0.0,None,None,1344584,Credit line increase/decrease,2015-04-23T12:00:00-05:00,CO,...,Closed with non-monetary relief,JPMORGAN CHASE & CO.,Web,2015-04-23T12:00:00-05:00,None,None,Yes,I have had several cards in good standing with...,None,Consent provided


In [11]:
(df.complaintwhathappened == "").sum()

57241

In [12]:
#Assign nan in place of blanks in the complaints column
df[df.complaintwhathappened == ""] = np.nan 

In [13]:
(df.complaintwhathappened == "").sum()

0

In [14]:
(df.complaintwhathappened == np.nan).sum()

0

In [15]:
#Remove all rows where complaints column is nan
df.dropna(subset=['complaintwhathappened'], inplace=True)

In [16]:
df.shape

(21072, 22)

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [17]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_text(text):
    # Make the text lowercase
    text = text.lower()
    
    # Remove text in square brackets using regular expression
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove punctuation using string library
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove words containing numbers using regular expression
    text = re.sub(r'\w*\d\w*', '', text)
    
    return text

In [18]:
#Write your function to Lemmatize the texts
def lemmatize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize each word using its part of speech (POS)
    lemmatized_words = []
    for word, pos in nltk.pos_tag(words):
        pos_letter = pos[0].lower() if pos[0].lower() in ['a', 'r', 'n', 'v'] else 'n'
        lemma = lemmatizer.lemmatize(word, pos=pos_letter)
        lemmatized_words.append(lemma)
    
    # Join the lemmatized words back into a sentence
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

In [19]:
df['cleaned_complaints'] = df['complaintwhathappened'].apply(clean_text)

In [20]:
df['lemmatized_complaints'] = df['cleaned_complaints'].apply(lemmatize_text)

In [21]:
df['lemmatized_complaints'].sample(10)

47877    cfpb please know that this be not a duplicate ...
70796    chase clear a fraudlent check they do not even...
11243    i recently apply for a chase credit card twice...
30077    chase bank refuse to add my spouse to my check...
19114    hello i be an authorized user on a relative ch...
8548     im a recent xxxx and im try to save up some mo...
22916    on unfortunately i deposit a fraudulent check ...
26292    in xxxx i apply for deferred payment with my l...
53843    i ask chase credit card service to close my ac...
413      in regard to xxxx chase card service can someo...
Name: lemmatized_complaints, dtype: object

In [22]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df_clean = pd.DataFrame(columns=['complaintwhathappened','lemmatized_complaints'], data=df[['complaintwhathappened','lemmatized_complaints']])

In [23]:
df_clean

,complaintwhathappened,lemmatized_complaints
1,Good morning my name is XXXX XXXX and I apprec...,good morning my name be xxxx xxxx and i apprec...
2,I upgraded my XXXX XXXX card in XX/XX/2018 and...,i upgrade my xxxx xxxx card in and be tell by ...
10,Chase Card was reported on XX/XX/2019. However...,chase card be report on however fraudulent app...
11,"On XX/XX/2018, while trying to book a XXXX XX...",on while try to book a xxxx xxxx ticket i come...
14,my grand son give me check for {$1600.00} i de...,my grand son give me check for i deposit it in...
...,...,...
78303,After being a Chase Card customer for well ove...,after be a chase card customer for well over a...
78309,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",on wednesday xxxxxxxx i call chas my xxxx xxxx...
78310,I am not familiar with XXXX pay and did not un...,i be not familiar with xxxx pay and do not und...
78311,I have had flawless credit for 30 yrs. I've ha...,i have have flawless credit for yr ive have ch...


In [30]:
#Write your function to extract the POS tags 

def get_pos_tag(text):
    words = []
    pos_tags = []
  # write your code here
    for word, pos in nltk.pos_tag(text):
        words.append(word)
        pos_tags.append(pos)
    
    return words, pos_tags

def remove_non_nn_words(text):
    words, pos_tags = get_pos_tag(text)
    sentence = ''
    
    for index, pos_tag in enumerate(pos_tags):
        if pos_tag == 'NN':
            sentence = sentence + ' ' + words[index]
    return sentence


In [31]:
df_clean["complaint_POS_removed"] =  df_clean['lemmatized_complaints'].apply(remove_non_nn_words)#this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].

KeyboardInterrupt: 

In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text


In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the unigram frequency


In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the bigram frequency

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the trigram frequency

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer 



#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.


## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = #write the value you want to test out

#keep the random_state =40
nmf_model = #write your code here

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics


In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4



In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = #write your code to assign topics to each rows.

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean=df_clean.groupby('Topic').head(5)
df_clean.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:

#Write your code to get the Vector count


#Write your code here to transform the word vector to tf-idf

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics



